In [ ]:
# === SUPPRESS OPENPYXL WARNINGS ===
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

# === STEP 1: LOAD THE NEW OUTBOUND DATA ===
# === INITIAL SETUP: RUN THIS CELL FIRST ===
import pandas as pd
import numpy as np

# Define the path to your new Inbound file
inbound_file_path = r"C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Inbound Report Data.xlsx" # <<< CHANGE THIS TO YOUR ACTUAL FILE PATH

# Read the new file into a DataFrame called 'inbound_df'
inbound_df = pd.read_excel(inbound_file_path)

# Let's see what we're working with
print("✅ Inbound Data Loaded Successfully!")
print(f"Shape: {inbound_df.shape} (rows, columns)")

In [ ]:
# Add new columns with default values
inbound_df['01_Inbound Type'] = ''
inbound_df['Inbound Qty (UOM)'] = 0
inbound_df['Inbound Qty > 8'] = ''
inbound_df['1H / 2H(Storage)'] = ''
inbound_df['Unstuffing Type'] = ''
inbound_df['Unstuffing'] = ''
inbound_df['SFA Booking'] = ''
inbound_df['SFA Status'] = ''
inbound_df['Sorting'] = ''
inbound_df['Date_PalletID_Location'] = ''
inbound_df['Storage Type'] = ''
inbound_df['Remarks'] = ''

In [ ]:
# === APPLY EXCEL FORMULA: REFERENCE D2, RESULT IN AC2 ===

# First, create a clean series by trimming whitespace from the D2 column (the reference)
clean_series = inbound_df['PO No'].str.strip()  # TRIM values from D2 column

# Normalize: trim spaces + lowercase for case-insensitive checks
clean_series_norm = clean_series.str.strip().str.lower()

conditions = [
    clean_series_norm.str.startswith('po'),
    clean_series_norm.str.startswith('to'),
    clean_series_norm.str.startswith('sro'),
    clean_series_norm.str.startswith('for '),
    clean_series_norm.str.startswith('ret'),
    clean_series_norm.str.startswith('order'),
    clean_series_norm == ''
]

choices = [
    'Receiving',
    'Plant Transfer', 
    'Return',
    'Return',
    'Return',
    'Return',
    '-'  # This handles the empty string case
]

# Apply the conditions and put the result in AC2 column
inbound_df['01_Inbound Type'] = np.select(conditions, choices, default='Import')

print("✅ Excel formula applied! Reference: PO No, Result: 01_Inbound Type")
print(inbound_df[['PO No', '01_Inbound Type']].head(15))

In [ ]:
import pandas as pd
import numpy as np

# === PART 1: Update 'Inbound Qty (UOM)' for UOM == 'CTN' ===
# This sets the value for rows where UOM is 'CTN'
inbound_df.loc[inbound_df['UOM'] == 'CTN', 'Inbound Qty (UOM)'] = inbound_df.loc[inbound_df['UOM'] == 'CTN', 'Qty Received']
print("✅ Step 1 Complete: 'Inbound Qty (UOM)' updated for UOM = 'CTN'")

# === PART 2: OVERRIDE 'Inbound Qty (UOM)' for specific Articles ===
# These rules will OVERWRITE the values for these specific articles,
# even if they were set by the UOM rule above.

# Now apply each rule one by one using .loc
inbound_df.loc[inbound_df['Article'] == 'ST-C730000002', 'Inbound Qty (UOM)'] = inbound_df['Qty Received'] / 10
inbound_df.loc[inbound_df['Article'] == 'ST-C230000165', 'Inbound Qty (UOM)'] = np.nan
inbound_df.loc[inbound_df['Article'] == 'ST-C730000093', 'Inbound Qty (UOM)'] = inbound_df['Qty Received'] / 5
inbound_df.loc[inbound_df['Article'] == 'ST-C730000023', 'Inbound Qty (UOM)'] = inbound_df['Qty Received'] / 5
inbound_df.loc[inbound_df['Article'] == 'ST-C730000226', 'Inbound Qty (UOM)'] = inbound_df['Qty Received']
inbound_df.loc[inbound_df['Article'] == 'ST-C730000021', 'Inbound Qty (UOM)'] = inbound_df['Qty Received'] / 10
inbound_df.loc[inbound_df['Article'] == 'ST-C230000534', 'Inbound Qty (UOM)'] = np.nan

print("✅ Step 2 Complete: 'Inbound Qty (UOM)' updated based on Article rules.")

# === FINAL CHECK: Print a sample to verify the column ===
print("\n🔍 Final Check - Sample of relevant columns:")
print(inbound_df[['Article', 'UOM', 'Qty Received', 'Inbound Qty (UOM)']].head(15))

In [ ]:
# === CONVERT EXCEL FORMULA: SUMIF PO No > 8 ===

# Calculate the total quantity for each PO No
po_totals = inbound_df.groupby('PO No')['Inbound Qty (UOM)'].transform('sum')

# Create the condition: if total for this PO No > 8, then True, else False
inbound_df['Inbound Qty > 8'] = po_totals > 8

# Convert boolean True/False to string "TRUE"/"FALSE" to match Excel
inbound_df['Inbound Qty > 8'] = inbound_df['Inbound Qty > 8'].map({True: 'TRUE', False: 'FALSE'})

print("✅ Excel SUMIF formula converted!")
print(inbound_df[['PO No', 'Inbound Qty (UOM)', 'Inbound Qty > 8']].head(15))

In [ ]:
# === CONVERT EXCEL FORMULA: DATE TO 1H/2H CATEGORY ===

# Convert the 'Receiving' column to datetime with DAY FIRST format (DD/MM/YYYY)
inbound_df['Receiving'] = pd.to_datetime(inbound_df['Receiving'], dayfirst=True)

# Extract the day of the month and apply the logic
inbound_df['1H / 2H(Storage)'] = np.where(
    inbound_df['Receiving'].dt.day <= 15,  # If day <= 15
    '1H',                                  # Then "1H"
    '2H'                                   # Else "2H"
)

print("✅ Excel date formula converted!")
print(inbound_df[['Receiving', '1H / 2H(Storage)']].head(15))


    Receiving 1H / 2H(Storage)
0  2025-08-01               1H
1  2025-08-01               1H
2  2025-08-01               1H
3  2025-08-01               1H
4  2025-08-01               1H
5  2025-08-01               1H
6  2025-08-01               1H
7  2025-08-01               1H
8  2025-08-01               1H
9  2025-08-01               1H
10 2025-08-01               1H
11 2025-08-01               1H
12 2025-08-01               1H
13 2025-08-01               1H
14 2025-08-01               1H


In [ ]:
# === STEP 1: LOAD PRODUCT DETAIL LISTING ===

product_detail_path = r"C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Product Detail Listing.xlsx"

# Read the product detail file into a DataFrame
product_detail_df = pd.read_excel(product_detail_path)

# Let's see what we're working with
print("✅ Product Detail Listing Loaded Successfully!")
print(f"Shape: {product_detail_df.shape} (rows, columns)")
print("\nFirst 5 rows:")
print(product_detail_df.head().to_string())

In [ ]:
# === STEP 2: UPDATE EXISTING 'Storage Type' COLUMN ===

# Normalize both keys to uppercase for case-insensitive matching
product_detail_df['Product Code Clean'] = product_detail_df['Product Code'].astype(str).str.strip().str.upper()
inbound_df['Article Clean'] = inbound_df['Article'].astype(str).str.strip().str.upper()

# Create a mapping series from the cleaned product detail: Product Code -> Product Category
storage_mapping = product_detail_df.set_index('Product Code Clean')['Product Category']

# Map using the cleaned Article, but store result in original 'Storage Type' column
inbound_df['Storage Type'] = inbound_df['Article Clean'].map(storage_mapping)

# Drop the temporary helper column if you don’t need it later
inbound_df = inbound_df.drop(columns=['Article Clean'])

print("✅ Mapping complete! 'Storage Type' column updated from Product Detail Listing (case-insensitive).")
print(inbound_df[['Article', 'Storage Type']].head(15))


In [ ]:
# === CONDITIONAL CONCATENATION BASED ON STORAGE TYPE ===

# First, ensure both columns are strings for concatenation
inbound_df['Receiving'] = inbound_df['Receiving'].astype(str)
inbound_df['Pallet ID'] = inbound_df['Pallet ID'].astype(str)
inbound_df['Location'] = inbound_df['Location'].astype(str)

# Apply different concatenation logic based on Storage Type
inbound_df['Date_PalletID_Location'] = np.where(
    inbound_df['Storage Type'] == 'FROZEN',
    inbound_df['Receiving'] + inbound_df['Pallet ID'],      # For FROZEN: Receiving + Pallet ID
    inbound_df['Receiving'] + inbound_df['Location']        # For others (CHILLED): Receiving + Location
)

print("✅ Conditional concatenation complete!")
print(inbound_df[['Storage Type', 'Receiving', 'Pallet ID', 'Location', 'Date_PalletID_Location']].head(15))

In [ ]:
# === STEP 3: CLEAN UP (OPTIONAL) ===

# Drop the 'Product Code' column if you don't need it
inbound_df.drop(columns=['Product Code'], inplace=True, errors='ignore')

print("✅ Clean up complete!")

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill

# --- File Paths ---
# CORRECTED PATH: Removed extra space and fixed folder structure
sfa_status_path = r"C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Products SFA Status 2025.xlsx"
modified_inbound_path = r"C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Result\Modified_Inbound_Data.xlsx"

# ==================== STEP 1: USE EXISTING inbound_df ====================

print("📖 Using existing inbound_df for processing...")

# Verify we have the expected columns in inbound_df
expected_columns = ['01_Inbound Type', 'Inbound Qty (UOM)', 'Inbound Qty > 8', '1H / 2H(Storage)', 
                   'Unstuffing Type', 'Unstuffing', 'SFA Booking', 'SFA Status', 'Sorting', 
                   'Date_PalletID_Location', 'Storage Type', 'Remarks']

print("📋 Checking expected columns in inbound_df:")
for col in expected_columns:
    if col in inbound_df.columns:
        print(f"   ✅ '{col}' exists")
    else:
        print(f"   ❌ '{col}' missing - adding now")
        inbound_df[col] = ''  # Add missing column with empty values

# ==================== STEP 2: SFA STATUS LOOKUP ====================

print("🔍 Finding the correct sheet in SFA Status file...")
try:
    xl_sfa = pd.ExcelFile(sfa_status_path)
    sfa_sheet_name = None

    for sheet in xl_sfa.sheet_names:
        df_check = pd.read_excel(sfa_status_path, sheet_name=sheet, nrows=5, header=None)
        if not df_check.empty and len(df_check.columns) > 4 and df_check.iat[2, 4] == 'PO Number':
            sfa_sheet_name = sheet
            print(f"   Found target sheet: '{sfa_sheet_name}'")
            break

    if sfa_sheet_name is None:
        raise ValueError("❌ Could not find a sheet where cell E3 contains 'PO Number'. Please check the file.")

    print("📖 Loading SFA Status data...")
    df_sfa = pd.read_excel(sfa_status_path, sheet_name=sfa_sheet_name, header=2)

    # Create mapping with cleaned PO numbers
    if 'SFA Status' in df_sfa.columns:
        df_sfa_clean = df_sfa[['PO Number', 'SFA Status']].dropna()
        df_sfa_clean['PO Number Clean'] = df_sfa_clean['PO Number'].astype(str).str.strip().str.upper()
        sfa_status_map = df_sfa_clean.set_index('PO Number Clean')['SFA Status']
        print("✅ Using 'SFA Status' column with cleaned PO numbers")
    else:
        df_sfa_clean = df_sfa[['PO Number', df_sfa.columns[7]]].dropna()
        df_sfa_clean['PO Number Clean'] = df_sfa_clean['PO Number'].astype(str).str.strip().str.upper()
        sfa_status_map = df_sfa_clean.set_index('PO Number Clean').iloc[:, 0]
        print("✅ Using column H with cleaned PO numbers")

    print(f"   Loaded SFA status for {len(sfa_status_map)} POs.")

    # Find the PO column in inbound data
    po_column = None
    for col in inbound_df.columns:
        if 'po' in str(col).lower() or 'PO' in str(col):
            po_column = col
            break

    if po_column is None:
        raise ValueError("❌ Could not find PO column in inbound data")

    print(f"✅ Using '{po_column}' as PO column")

    # Clean the PO numbers in the inbound data
    inbound_df['PO No Clean'] = inbound_df[po_column].astype(str).str.strip().str.upper()

    print("⚙️  Processing SFA Status lookup...")

    def get_sfa_booking_status(po_number_clean):
        if pd.notna(po_number_clean) and po_number_clean in sfa_status_map.index:
            original_value = str(sfa_status_map[po_number_clean]).strip()
            original_value_lower = original_value.lower()
            
            if 'waive' in original_value_lower:
                return ('SFA', 'Waived')
            elif 'pass' in original_value_lower:
                return ('SFA', 'Passed')
            else:
                return (None, None)
        else:
            return (None, None)

    # Apply SFA lookup to populate SFA Booking and SFA Status columns
    update_count = 0
    passed_count = 0
    waived_count = 0

    for index, row in inbound_df.iterrows():
        po_number_clean = row['PO No Clean']
        booking_val, status_val = get_sfa_booking_status(po_number_clean)
        
        if booking_val is not None:
            inbound_df.at[index, 'SFA Booking'] = booking_val
            inbound_df.at[index, 'SFA Status'] = status_val
            update_count += 1
            
            if status_val == 'Passed':
                passed_count += 1
            elif status_val == 'Waived':
                waived_count += 1

    # Remove temporary column
    inbound_df = inbound_df.drop('PO No Clean', axis=1)
    print(f"✅ Updated SFA info for {update_count} records.")
    print(f"   - Passed: {passed_count}")
    print(f"   - Waived: {waived_count}")

except FileNotFoundError:
    print("❌ SFA Status file not found. Skipping SFA lookup...")
    print(f"   Expected path: {sfa_status_path}")

# ==================== STEP 3: SORTING LOGIC ====================

print("⚙️  Calculating sorting requirements...")

# Filter for Import rows only
df_import = inbound_df[inbound_df["01_Inbound Type"] == "Import"].copy()

# Find required columns
article_col = next((col for col in inbound_df.columns if 'article' in str(col).lower()), 'Article')
expiry_col = next((col for col in inbound_df.columns if 'expiry' in str(col).lower()), 'Expiry Date')
remark_col = next((col for col in inbound_df.columns if 'invt' in str(col).lower() or 'remark' in str(col).lower()), 'Invt Remark')

print(f"✅ Using columns: PO='{po_column}', Article='{article_col}', Expiry='{expiry_col}', Remark='{remark_col}'")

# Build concatenated key and calculate unique combinations
df_import["concat_key"] = (
    df_import[po_column].astype(str).fillna("")
    + "_" + df_import[article_col].astype(str).fillna("")
    + "_" + df_import[expiry_col].astype(str).fillna("")
    + "_" + df_import[remark_col].astype(str).fillna("")
)

unique_counts = df_import.groupby(po_column)["concat_key"].nunique()

# Populate Sorting column
inbound_df["Sorting"] = ""
inbound_df.loc[inbound_df[po_column].isin(unique_counts[unique_counts > 5].index), "Sorting"] = "Sort"

sort_count = inbound_df["Sorting"].value_counts().get("Sort", 0)
print(f"✅ Marked {sort_count} POs for sorting")

# ==================== STEP 4: SAVE PROCESSED DATA ====================

print("💾 Saving processed data to Modified_Inbound_Data.xlsx...")
inbound_df.to_excel(modified_inbound_path, index=False, engine='openpyxl')
print("✅ Data processing completed successfully!")

# ==================== STEP 5: APPLY FORMATTING ====================

print("🎨 Applying formatting to headers...")
columns_to_style = ['01_Inbound Type', 'Inbound Qty (UOM)', 'Inbound Qty > 8', '1H / 2H(Storage)',
                   'Unstuffing Type', 'Unstuffing', 'SFA Booking', 'SFA Status', 'Sorting',
                   'Date_PalletID_Location', 'Storage Type', 'Remarks']

wb = load_workbook(modified_inbound_path)
ws = wb.active

light_yellow_fill = PatternFill(start_color='FFFFD6', end_color='FFFFD6', fill_type='solid')
header_font = Font(name='Calibri', size=11, color='800000', bold=True)

for col_idx, cell in enumerate(ws[1], 1):
    if cell.value in columns_to_style:
        cell.fill = light_yellow_fill
        cell.font = header_font

wb.save(modified_inbound_path)

print(f"✅ SUCCESS! Formatted file saved to: {modified_inbound_path}")
print(f"✅ Applied formatting to headers of key columns")
print(f"✅ All data processing completed successfully!")

   Found target sheet: 'Master '
📖 Loading SFA Status data...


✅ Using 'SFA Status' column with cleaned PO numbers
   Loaded SFA status for 102 POs.
✅ Using 'PO No' as PO column
⚙️  Processing SFA Status lookup...


✅ Updated SFA info for 440 records.
   - Passed: 43
   - Waived: 397
⚙️  Calculating sorting requirements...
✅ Using columns: PO='PO No', Article='Article', Expiry='Expiry Date', Remark='Remarks'
✅ Marked 48 POs for sorting
💾 Saving processed data to Modified_Inbound_Data.xlsx...


✅ Data processing completed successfully!
🎨 Applying formatting to headers...


✅ SUCCESS! Formatted file saved to: C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Result\Modified_Inbound_Data.xlsx
✅ Applied formatting to headers of key columns
✅ All data processing completed successfully!


In [ ]:
# === NEW CELL: ADD UNSTUFFING LOGIC TO EXISTING MODIFIED_INBOUND_DATA ===

import pandas as pd

# File paths
modified_inbound_path = r"C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Result\Modified_Inbound_Data.xlsx"

print("📖 Loading existing Modified_Inbound_Data.xlsx...")
# Load the already processed data
df_modified = pd.read_excel(modified_inbound_path, engine='openpyxl')

# FIX: Convert Unstuffing columns to string type first
print("⚙️  Converting Unstuffing columns to string type...")
df_modified['Unstuffing Type'] = df_modified['Unstuffing Type'].astype(str)
df_modified['Unstuffing'] = df_modified['Unstuffing'].astype(str)

# Replace 'nan' strings with empty string
df_modified['Unstuffing Type'] = df_modified['Unstuffing Type'].replace('nan', '', regex=False)
df_modified['Unstuffing'] = df_modified['Unstuffing'].replace('nan', '', regex=False)

print("🔍 Analyzing Import records for Unstuffing logic...")

# Filter only Import records
df_import = df_modified[df_modified["01_Inbound Type"] == "Import"].copy()

# Group by PO No and count unique Date_PalletID_Location for each PO
unique_counts = df_import.groupby('PO No')['Date_PalletID_Location'].nunique()

print(f"📊 Found {len(unique_counts)} unique PO Numbers with Import type")

# Apply Unstuffing logic based on unique count
for po_number, unique_count in unique_counts.items():
    if unique_count > 24:
        # If > 24 unique locations, use 40 RF Container
        df_modified.loc[(df_modified['PO No'] == po_number) & (df_modified['01_Inbound Type'] == 'Import'), 
                       'Unstuffing Type'] = '40 RF Container'
        df_modified.loc[(df_modified['PO No'] == po_number) & (df_modified['01_Inbound Type'] == 'Import'), 
                       'Unstuffing'] = 'Loose'
    else:
        # If <= 24 unique locations, use 20 RF Container
        df_modified.loc[(df_modified['PO No'] == po_number) & (df_modified['01_Inbound Type'] == 'Import'), 
                       'Unstuffing Type'] = '20 RF Container'
        df_modified.loc[(df_modified['PO No'] == po_number) & (df_modified['01_Inbound Type'] == 'Import'), 
                       'Unstuffing'] = 'Loose'

# Count how many POs got each Unstuffing Type
forty_count = (df_modified['Unstuffing Type'] == '40 RF Container').sum()
twenty_count = (df_modified['Unstuffing Type'] == '20 RF Container').sum()

print(f"✅ Unstuffing logic applied!")
print(f"   - 40 RF Container (Loose): {forty_count} records")
print(f"   - 20 RF Container (Loose): {twenty_count} records")
print(f"   - Total Import records processed: {len(df_import)}")

# Save back to the same file
print("💾 Saving updated data back to Modified_Inbound_Data.xlsx...")
df_modified.to_excel(modified_inbound_path, index=False, engine='openpyxl')

print("✅ Unstuffing logic completed successfully!")
print("📋 Sample of updated records:")
sample_df = df_modified[df_modified['01_Inbound Type'] == 'Import'][['PO No', '01_Inbound Type', 'Date_PalletID_Location', 'Unstuffing Type', 'Unstuffing']].head(10)
print(sample_df.to_string(index=False))

⚙️  Converting Unstuffing columns to string type...
🔍 Analyzing Import records for Unstuffing logic...
📊 Found 25 unique PO Numbers with Import type
✅ Unstuffing logic applied!
   - 40 RF Container (Loose): 1057 records
   - 20 RF Container (Loose): 0 records
   - Total Import records processed: 1057
💾 Saving updated data back to Modified_Inbound_Data.xlsx...


✅ Unstuffing logic completed successfully!
📋 Sample of updated records:
      PO No 01_Inbound Type Date_PalletID_Location Unstuffing Type Unstuffing
CGMU5171930          Import 2025-08-01CFS20100116N 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20100707T 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20101151N 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20101202K 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20101652T 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20101875- 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20102333Q 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20102386Y 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20102599/ 40 RF Container      Loose
CGMU5171930          Import 2025-08-01CFS20102820R 40 RF Container      Loose


In [ ]:
# ==================== STEP 5: APPLY FORMATTING ====================

print("🎨 Applying formatting to headers...")
columns_to_style = ['01_Inbound Type', 'Inbound Qty (UOM)', 'Inbound Qty > 8', '1H / 2H(Storage)',
                   'Unstuffing Type', 'Unstuffing', 'SFA Booking', 'SFA Status', 'Sorting',
                   'Date_PalletID_Location', 'Storage Type', 'Remarks']

wb = load_workbook(modified_inbound_path)
ws = wb.active

light_yellow_fill = PatternFill(start_color='FFFFD6', end_color='FFFFD6', fill_type='solid')
header_font = Font(name='Calibri', size=11, color='800000', bold=True)

for col_idx, cell in enumerate(ws[1], 1):
    if cell.value in columns_to_style:
        cell.fill = light_yellow_fill
        cell.font = header_font

wb.save(modified_inbound_path)

print(f"✅ SUCCESS! Formatted file saved to: {modified_inbound_path}")
print(f"✅ Applied formatting to headers of key columns")

✅ SUCCESS! Formatted file saved to: C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Result\Modified_Inbound_Data.xlsx
✅ Applied formatting to headers of key columns
